# Project 1 Temp

### Feature Selection

Temp text here

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline
# Above is directly taken from worksheet 3
print('Import libraries successful!')

Import libraries successful!


In [ ]:
# import data the standard way - better to use panda?
# trainFile = open('train_tweets.txt',  encoding="utf8")
# train = trainFile.read()

# testFile = open('test_tweets_unlabeled.txt',  encoding="utf8")
# test = testFile.read()
# print('Import files successful!')

In [ ]:
# # Attempt to vectorise - failed
# from sklearn.feature_extraction import DictVectorizer
# vec = DictVectorizer()
# vec.fit_transform(train).toarray()

In [2]:
# Import file using panda - seperated by tab and transforfmed to CSV to allow for features
# The .head() just prints off the first few lines of the array, except it seems to be overridden 
# by the print statement
data = pd.read_csv('train_tweets.txt', sep="\t", header=None)
data.columns = ['id','tweet']
print('Import and shift successful in panda!')

#Number of words
data['word_count'] = data['tweet'].apply(lambda x: len(str(x).split(" ")))
# data[['tweet','word_count']].head()
print('Adding length of tweets!')

# Number of charecters
data['char_count'] = data['tweet'].str.len() ## this also includes spaces
data[['tweet','char_count']].head()
print('Adding the number of charecters in a tweet')
# print(data)

Import and shift successful in panda!
Adding length of tweets!
Adding the number of charecters in a tweet


In [ ]:
# Average word length
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

data['avg_word'] = data['tweet'].apply(lambda x: avg_word(x))
data[['tweet','avg_word']].head()
print('Added average words!')

In [ ]:
# - run this if the first time in anaconda
import nltk
nltk.download('stopwords')

In [ ]:
# https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
# Removing stopwords (the,a,we,can,..)
# the stopwords are quite aggressive so might omit or will create a seperate version
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
stop = stopwords.words('english')
# stop = set(stopwords.words('english'))
# Get the number of stop words
data['stopwords'] = data['tweet'].apply(lambda x: len([x for x in x.split() if x in stop]))
data[['tweet','stopwords']].head()

In [ ]:
# from nltk.tokenize import TweetTokenizer
# tknzr = TweetTokenizer()
# s0 = "This is a cooool #dummysmiley: :-) :-P <3 and some arrows < > -> <--"
# tknzr.tokenize(s0)

In [ ]:
# Number of special charecters
data['hastags'] = data['tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
data[['tweet','hastags']].head()

In [ ]:
# Number of numerics
data['numerics'] = data['tweet'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
data[['tweet','numerics']].head()

In [ ]:
# UPPERCASE words
data['upper'] = data['tweet'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
data[['tweet','upper']].head()

### PRE-PROCESSING

In [4]:
# Converts all tweets into lowercase to avoid multiple copies of words
data['tweet'] = data['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data['tweet'].head()

0       @handle let's try and catch up live next week!
1    going to watch grey's on the big screen - thur...
2    @handle my pleasure patrick....hope you are well!
3    @handle hi there! been traveling a lot and lot...
4    rt @handle looking to drink clean & go green? ...
Name: tweet, dtype: object

In [5]:
# Removes all punctuation - reduces training data
data['tweet'] = data['tweet'].str.replace('[^\w\s]','')
data['tweet'].head()

0          handle lets try and catch up live next week
1    going to watch greys on the big screen  thursd...
2          handle my pleasure patrickhope you are well
3    handle hi there been traveling a lot and lots ...
4    rt handle looking to drink clean  go green pur...
Name: tweet, dtype: object

In [6]:
# Removing stop words
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data['tweet'].head()

0                 handle lets try catch live next week
1     going watch greys big screen thursday indulgence
2                     handle pleasure patrickhope well
3    handle hi traveling lot lots come next month r...
4    rt handle looking drink clean go green purchas...
Name: tweet, dtype: object

In [7]:
# Removing the 10 most common words - do before or after stop words? (It removes handle...)
freq = pd.Series(' '.join(data['tweet']).split()).value_counts()[:10]
freq = list(freq.index)
data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
data['tweet'].head()

0                        lets try catch live next week
1     going watch greys big screen thursday indulgence
2                            pleasure patrickhope well
3    hi traveling lot lots come next month recovere...
4    looking drink clean go green purchase clear2go...
Name: tweet, dtype: object

In [8]:
# 10 least common words - worth removing??
inFreq = pd.Series(' '.join(data['tweet']).split()).value_counts()[-10:]

In [9]:
# Spelling correction - works now
# Need to install textblob via conda -> run the below statement in commandprompt/terminal
# conda install -c conda-forge textblob
from textblob import TextBlob
data['tweet'][:5].apply(lambda x: str(TextBlob(x).correct()))

0                        lets try catch live next week
1      going watch grey big screen thursday indulgence
2                            pleasure patrickhope well
3    hi traveling lot lots come next month recovere...
4    looking drink clean go green purchase clear2go...
Name: tweet, dtype: object

In [10]:
# Lemmenisation - works now
# run below download if it is the first time running
# import nltk
# nltk.download('wordnet')

from textblob import Word
data['tweet'] = data['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['tweet'].head()

0                         let try catch live next week
1      going watch grey big screen thursday indulgence
2                            pleasure patrickhope well
3    hi traveling lot lot come next month recovered...
4    looking drink clean go green purchase clear2go...
Name: tweet, dtype: object

### Advanced processing

## Project Testing

In [11]:
# Bag of words
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(data['tweet'])

In [12]:
# Split our LABELED training data (80/20)
from sklearn.model_selection import train_test_split

train_bow = bow[:328195,:]
test_bow = bow[328195:,:]

X_train, X_test, y_train, y_test = train_test_split(train_bow, data['id'], test_size=0.10, random_state=42)
print("Training set has {} instances. Test set has {} instances.".format(X_train.shape[0], X_test.shape[0]))
# features = ['stopwords']
# x = X_train[features].values.ravel()
# x_test = X_test[features].values.ravel()
# y = y_train

Training set has 295375 instances. Test set has 32820 instances.


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
lreg = LogisticRegression(tol=0.1)
# lreg.fit(X_train, y_train)

# prediction = lreg.predict_proba(X_test) # predicting on the validation set
# prediction_int = prediction[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 than 1 else 0
# prediction_int = prediction_int.astype(np.int)

# f1_score(y_test, prediction_int) # calculating f1 score

In [14]:
# TF-IDF Features
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(data['tweet'])
print(tfidf)

  (0, 487)	0.4029702449636645
  (0, 888)	0.4595095707564396
  (0, 136)	0.5491106012964109
  (0, 499)	0.41379132449598516
  (0, 944)	0.39207170692732896
  (1, 358)	0.4192822537381518
  (1, 930)	0.48740297801705434
  (1, 88)	0.4836873179909417
  (1, 854)	0.5938748244664801
  (3, 407)	0.4289352032343252
  (3, 511)	0.7431450449048352
  (3, 171)	0.3261312140797652
  (3, 562)	0.39672215070824707
  (4, 506)	0.42752015857187725
  (4, 247)	0.5118130229162409
  (4, 159)	0.5633274431421991
  (4, 369)	0.4877869774716098
  (5, 336)	0.5893639940772505
  (5, 417)	0.591553840042965
  (5, 596)	0.5501946353934513
  (6, 346)	0.7557917545557259
  (6, 525)	0.6548120522299337
  (7, 447)	1.0
  (8, 850)	0.5645751428517962
  (8, 335)	0.6312517170075768
  :	:
  (328185, 687)	0.42396368258468153
  (328185, 812)	0.46966600498554084
  (328186, 914)	1.0
  (328187, 556)	0.4447918911303758
  (328187, 838)	0.8956339506654848
  (328188, 973)	0.6281184846323796
  (328188, 687)	0.7781177091309021
  (328189, 287)	0.743419

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

train_tfidf = tfidf[:328195,:]
test_tfidf = tfidf[328195:,:]

xtrain_tfidf = train_tfidf[y_train.index]
xvalid_tfidf = train_tfidf[y_test.index]

# lreg = LogisticRegression(tol=0.1)
# lreg.fit(xtrain_tfidf, y_train)

# prediction = lreg.predict_proba(xvalid_tfidf)
# prediction_int = prediction[:,1] >= 0.3
# prediction_int = prediction_int.astype(np.int)

# f1_score(yvalid, prediction_int)

In [16]:
# from sklearn.datasets import load_iris
# from sklearn.linear_model import LogisticRegression
# X, y = load_iris(return_X_y=True)
# clf = LogisticRegression(random_state=0, solver='lbfgs',
#                           multi_class='multinomial').fit(X, y)
# print(len(y))